In [1]:
import time
import sys
import os
import pandas as pd

In [2]:
import requests

res = requests.get("https://support.sas.com/documentation/onlinedoc/viya/exampledatasets/hmeq.csv")

with open("/workspaces/myfolder/Propensity/Data/hmeq_py_example.csv","w") as f:
    f.write(res.text)

In [3]:
from sklearn.metrics import accuracy_score, roc_curve
from sklearn.model_selection import train_test_split
from sasviya.ml.tree import ForestClassifier
from matplotlib import pyplot as plt
import sklearn
from sklearn.ensemble import RandomForestClassifier

In [4]:
## Load data, impute and then display sample
input_df = pd.read_csv('/workspaces/myfolder/Propensity/Data/autoloan.csv').dropna()
class_inputs = "JobType"
target = "BAD"

In [5]:
while len(input_df) < 100_000:
    input_df = pd.concat([input_df, input_df], ignore_index=True)

In [6]:
# Drop CustomerID and JobType. Not using JobType to make the workflow example easier without worrying about dummying.
df_all_inputs = input_df.drop([target, "REASON", "JOB","APPDATE","CITY","STATE","DIVISION","REGION"], axis=1)
df_all_inputs


,LOAN,LOANDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,1700.0,3054.8,4032.0,9.0,0.0,0.0,101.466002,1.0,8.0,37.113614
1,1800.0,2850.2,4303.4,11.0,0.0,0.0,88.766030,0.0,8.0,36.884894
2,2300.0,10237.0,12095.3,2.0,0.0,0.0,90.992533,0.0,13.0,31.588503
3,2400.0,3486.3,4747.1,12.0,0.0,0.0,70.491080,1.0,21.0,38.263601
4,2400.0,9844.9,11719.5,4.0,0.0,0.0,93.811775,0.0,13.0,29.681827
...,...,...,...,...,...,...,...,...,...,...
107323,88900.0,5726.4,9018.5,16.0,0.0,0.0,221.808718,0.0,16.0,36.112347
107324,89000.0,5457.6,9293.7,16.0,0.0,0.0,208.692070,0.0,15.0,35.859971
107325,89200.0,5404.5,9292.4,15.0,0.0,0.0,212.279697,0.0,15.0,35.556590
107326,89800.0,5037.0,9186.1,14.0,0.0,0.0,213.892709,0.0,16.0,34.340882


In [7]:
# Create training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(
     df_all_inputs, input_df[target], test_size=0.33, random_state=54321)

In [8]:
%%timeit
rf = RandomForestClassifier(n_estimators=100, 
                            max_depth=5,
                            min_samples_leaf=1,
                            max_features=None, 
                            criterion='gini',
                            random_state=0)
rf.fit(X_train, y_train)

KeyboardInterrupt: 

In [25]:
%%timeit
rf_job = RandomForestClassifier(n_jobs=-1,
                            n_estimators=100, 
                            max_depth=5,
                            min_samples_leaf=1,
                            max_features=None, 
                            criterion='gini',
                            random_state=0)
rf_job.fit(X_train, y_train)

4.27 s ± 36.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%%timeit
sasrf = ForestClassifier(n_estimators=100, 
                            max_depth=5, 
                            min_samples_leaf=1,
                            max_features=None, 
                            criterion='gini',
                            random_state=0)
sasrf.fit(X_train, y_train)

2.85 s ± 91.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
